In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
#reading DPMU phase A output from gridlabD
dpmu_phaseA = pd.read_csv('multi_voltageA.csv')
dpmu_phaseB = pd.read_csv('multi_voltageB.csv')
dpmu_phaseC = pd.read_csv('multi_voltageC.csv')


In [ ]:
column_names = ["NodeId", "realA", "imagA", "realB", "imagB", "realC", "imagC"]

In [ ]:
dpmu_data = pd.DataFrame(columns=column_names)

In [ ]:
index = 0
for (nodeName, nodePhaseA) in dpmu_phaseA.iteritems():
    if nodeName == '# timestamp' or nodeName == 'Node650' or nodeName == 'Node632' or nodeName == 'Node633' or nodeName == 'Node684' or nodeName == 'Node680':
        continue
    nodePhaseA_values = pd.Series(nodePhaseA.values).str.replace("i", "j").apply(lambda x: complex(x))
    
    for val in nodePhaseA_values:
            dpmu_data.loc[index, 'NodeId'] = nodeName
            dpmu_data.loc[index, 'realA'] = val.real
            dpmu_data.loc[index, 'imagA'] = val.imag
            index = index+1
             

In [ ]:
index = 0
for (nodeName, nodePhaseB) in dpmu_phaseB.iteritems():
    if nodeName == '# timestamp' or nodeName == 'Node650' or nodeName == 'Node632' or nodeName == 'Node633' or nodeName == 'Node684' or nodeName == 'Node680':
        continue
    nodePhaseB_values = pd.Series(nodePhaseB.values).str.replace("i", "j").apply(lambda x: complex(x))
    
    for val in nodePhaseB_values:
            dpmu_data.loc[index, 'NodeId'] = nodeName
            dpmu_data.loc[index, 'realB'] = val.real
            dpmu_data.loc[index, 'imagB'] = val.imag
            index = index+1

In [ ]:
index = 0
for (nodeName, nodePhaseC) in dpmu_phaseC.iteritems():
    if nodeName == '# timestamp' or nodeName == 'Node650' or nodeName == 'Node632' or nodeName == 'Node633' or nodeName == 'Node684' or nodeName == 'Node680':
        continue
    nodePhaseC_values = pd.Series(nodePhaseC.values).str.replace("i", "j").apply(lambda x: complex(x))
    
    for val in nodePhaseC_values:
            dpmu_data.loc[index, 'NodeId'] = nodeName
            dpmu_data.loc[index, 'realC'] = val.real
            dpmu_data.loc[index, 'imagC'] = val.imag
            index = index+1


In [ ]:
dpmu_data['realA'] = pd.to_numeric(dpmu_data['realA'])
dpmu_data['realB'] = pd.to_numeric(dpmu_data['realB'])
dpmu_data['realC'] = pd.to_numeric(dpmu_data['realC'])
dpmu_data['imagA'] = pd.to_numeric(dpmu_data['imagA'])
dpmu_data['imagB'] = pd.to_numeric(dpmu_data['imagB'])
dpmu_data['imagC'] = pd.to_numeric(dpmu_data['imagC'])
print(dpmu_data.dtypes)

In [ ]:
player_file_column_names = ['NodeId', 'real', 'imag']
input_player_df = pd.DataFrame(columns=player_file_column_names)

In [ ]:
directory = r"C:\Users\pragn\Desktop\grid-edge\data\input-data\nonscaled_inputs"

index = 0
for filename in os.listdir(directory):
    player = pd.read_csv(os.path.join(directory, filename))
    player['grid'] = player['grid'].str.replace("i", "j").apply(lambda x: complex(x))
    node_id = player['id'].loc[0]
    for i in range(0, 1441):
        input_player_df.loc[index, 'NodeId'] = 'Node'+str(node_id)
        input_player_df.loc[index, 'real'] = player['grid'].loc[i].real
        input_player_df.loc[index, 'imag'] = player['grid'].loc[i].imag
        index = index+1


In [ ]:
input_player_df['real'] = pd.to_numeric(input_player_df['real'])
input_player_df['imag'] = pd.to_numeric(input_player_df['imag'])

In [ ]:
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression, SGDRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import xgboost as xgb
import joblib

# create datasets
node_array = dpmu_data['NodeId'].unique()
check = False

for node in node_array:
    print(node)
    X = input_player_df[input_player_df['NodeId'] == node].drop(['NodeId'], axis=1)
    y = dpmu_data[dpmu_data['NodeId']==node].drop(['NodeId'], axis=1)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)
    
    if check:
        model = joblib.load("linear_model.joblib")
    else:
        model = LinearRegression()
        check = True
    
    model.fit(X_train, y_train)
    joblib.dump(model, "linear_model.joblib")
    
    y_pred = model.predict(X_test)
    print(mean_absolute_error(y_test, y_pred))
